<a href="https://colab.research.google.com/github/Steboss/NLP_on_TPU/blob/master/GPT_2_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here we are trying to use gpt-2

In [1]:
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 209, done.
remote: Total 209 (delta 0), reused 0 (delta 0), pack-reused 209
Receiving objects: 100% (209/209), 4.37 MiB | 3.42 MiB/s, done.
Resolving deltas: 100% (109/109), done.


In [0]:
import os
os.chdir("gpt-2")

In [3]:
!ls

CONTRIBUTORS.md  Dockerfile.gpu     LICENSE	   requirements.txt
DEVELOPERS.md	 domains.txt	    model_card.md  src
Dockerfile.cpu	 download_model.py  README.md


In [4]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 120kB/s 
     |████████████████████████████████| 3.2MB 38.0MB/s 
     |████████████████████████████████| 368kB 43.7MB/s 
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0


In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 81kB 3.2MB/s 
     |████████████████████████████████| 604kB 10.5MB/s 
     |████████████████████████████████| 51kB 20.5MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=764d1a38a8efb5b423fcc50bc66017bf251f5998f3591bc8bf8ef4cb8ee81c83
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533185 sha256=628b0dd215d74a8aa1cff37d8db898bb6a144e4a1fd0850a125d2cac73f72392
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [6]:
!python download_model.py 774M

Fetching checkpoint: 1.00kit [00:00, 733kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 41.5Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 709kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [01:44, 29.6Mit/s]                                 
Fetching model.ckpt.index: 16.0kit [00:00, 8.23Mit/s]                                               
Fetching model.ckpt.meta: 1.38Mit [00:00, 36.6Mit/s]                                                
Fetching vocab.bpe: 457kit [00:00, 18.1Mit/s]                                                       


In [0]:
!export PYTHONIOENCODING=UTF-8

In [0]:
os.chdir("src")

In [0]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder

In [0]:
def interact_model(model_name,seed,nsamples,batch_size,length,temperature,top_k, models_dir):
  
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [12]:
interact_model('774M', 123456789, 1, 1, 300, 1, 0,'/content/gpt-2/models')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/gpt-2/models/774M/model.ckpt
Model prompt >>> This is my story...
======================================== SAMPLE 1 ========================================
I was born No.1 ...and I'm the favorite flag of 46% of Americans."

How many Americans actually want the Green Flag added to their flag?

Despite being small, with similar aesthetics, popularity and even a thumbs up or down consensus from 62.6% of respondents, the flag was quickly quashed by Clint Eastwood's Gran Torino and Hollywood conservatives.

In a suitably offensive lead-line and worst case to date, actor Samuel L. Jackson briefly made his mark with a suspiciously symbolic image of the flag. Cuban film dad Pedr

KeyboardInterrupt: ignored

In [13]:
#model_name,seed,nsamples,batch_size,length,temperature,top_k, models_dir):
interact_model('774M', 123456789, 1, 1,1000, 20, 40,'/content/gpt-2/models')

INFO:tensorflow:Restoring parameters from /content/gpt-2/models/774M/model.ckpt
Model prompt >>> It was a very long time ago
======================================== SAMPLE 1 ========================================

As this little bit on this small hill. At it it used all you could afford; as this old boy went home each of me did what it did so at last. A beautiful time with them at our dinner, just when I took your hand in return a tall blond, just the youngest and still the newest born man or maid for their children that were about all that would do when those great two did this kind visit as one; one, on which you saw for one your mother in front. And two were young me now just as this morning one who stood at home near one wall near each other the only room I now am in but no two like each little two room the last great two, just what two to this moment would think were there would sit there if to any place at present of our little table so it will you come if need they call there

KeyboardInterrupt: ignored